In [1]:
from env import config, BASE_DIR
from sqlalchemy import create_engine, text

In [35]:
import pandas as pd
import glob
import datetime

In [3]:
DATABASE_URL = config("FLIGHT_PRICES_DATABASE_URL", cast=str, default=None)
DATABASE_URL is not None

True

In [4]:
engine = create_engine(str(DATABASE_URL))

In [5]:
DATASET_DIR = BASE_DIR / "datasets"
DATASET_DIR.exists()

True

In [13]:
GRETEL_DIR = DATASET_DIR / "gretel"
GRETEL_DIR.exists()

True

In [9]:
table_name = "flight_prices"
df = pd.read_sql(table_name, engine, index_col="index")
df.head()

,flightDate,startingAirport,destinationAirport,isBasicEconomy,isRefundable,isNonStop,segmentsAirlineName,totalFare
index,,,,,,,,
0,2022-04-17,ATL,BOS,0,0,0,American Airlines||American Airlines,273
1,2022-04-17,ATL,BOS,0,0,0,Spirit Airlines||Spirit Airlines,301
2,2022-04-17,ATL,BOS,0,0,0,United||United,352
3,2022-04-17,ATL,BOS,0,0,1,American Airlines,355
4,2022-04-17,ATL,BOS,0,0,1,Delta,307


In [30]:
last_row = df.tail(1)
last_idx = last_row.index[0]

In [33]:
max_flight_date = df.flightDate.max()
max_flight_date

Timestamp('2022-04-22 00:00:00')

In [38]:
gretel_dfs = []
for path in glob.glob(str(GRETEL_DIR / '*.csv')):
    with open(path) as f:
        gretel_dfs.append(pd.read_csv(f))
new_df = pd.concat(gretel_dfs)
new_df.index = new_df.index + last_idx + 1
new_df["flightDate"] = pd.to_datetime(new_df["flightDate"])
new_df["flightDate"] = pd.to_datetime(new_df["flightDate"]).apply(lambda x: max_flight_date + (max_flight_date - x) + datetime.timedelta(days=1))
new_df.head()

,flightDate,startingAirport,destinationAirport,isBasicEconomy,isRefundable,isNonStop,segmentsAirlineName,totalFare
9592,2022-04-24,LAX,DEN,0,0,1,American Airlines||American Airlines,338
9593,2022-04-26,BOS,DEN,0,0,0,Frontier Airlines||Frontier Airlines,202
9594,2022-04-24,BOS,DEN,1,0,0,United||United,271
9595,2022-04-25,MIA,LAX,0,0,0,Spirit Airlines||Spirit Airlines,668
9596,2022-04-26,JFK,SFO,1,0,1,United,148


In [42]:
new_df.to_sql(table_name, engine, if_exists='append', index=True)

10000

In [43]:
df = pd.read_sql(table_name, engine, index_col="index")

In [44]:
df.shape

(19592, 8)

In [47]:
df.tail()

,flightDate,startingAirport,destinationAirport,isBasicEconomy,isRefundable,isNonStop,segmentsAirlineName,totalFare
index,,,,,,,,
14587,2022-04-27,DTW,OAK,0,0,0,Spirit Airlines||Spirit Airlines,598
14588,2022-04-26,ATL,LAX,1,0,0,American Airlines||American Airlines,136
14589,2022-04-27,DFW,DEN,0,0,1,Delta,106
14590,2022-04-25,EWR,DEN,0,0,1,Delta,112
14591,2022-04-27,JFK,OAK,0,0,0,United||Alaska Airlines||Alaska Airlines,885
